In [1]:
# Add project root to sys.path so 'src' is importable
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("Project root in sys.path:", project_root)
print("Current working directory:", os.getcwd())

Project root in sys.path: c:\Users\wbrya\OneDrive\Documents\GitHub\AI-CFO-FYP
Current working directory: c:\Users\wbrya\OneDrive\Documents\GitHub\AI-CFO-FYP\notebooks


In [2]:
# Cell 5: Import necessary classes and modules
import os
from dotenv import load_dotenv
import io # Import the io module for BytesIO

# Assuming your src directory is installable with __init__.py files
from src.llm.GeminiClient import GeminiClient
from src.services.FinancialDocParser import FinancialDocParser

# Load environment variables again (good practice in notebooks)
load_dotenv()

# Verify Gemini Key (Crucial for FinancialDocParser)
gemini_key = os.environ.get("GEMINI_API_KEY")
if gemini_key:
    print(f"Gemini API Key loaded: {gemini_key[:3]}...{gemini_key[-4:]}")
else:
    print("Gemini API Key not loaded. FinancialDocParser will likely fail.")
    # Consider stopping here if the key is missing

Gemini API Key loaded: AIz...KbyQ


In [3]:
# Cell 6: Define PDF path and test FinancialDocParser

# --- Configuration ---
# Make sure this path points to your test PDF
TEST_PDF_PATH = "../data/source_pdfs/10k_tesla_3_pages.pdf" # Adjust filename as needed

# --- End Configuration ---

# Check if the test PDF file exists
if not os.path.exists(TEST_PDF_PATH):
    print(f"Error: Test PDF not found at {TEST_PDF_PATH}")
    print("Please update TEST_PDF_PATH to point to a valid PDF file in your data folder.")
else:
    print(f"Found test PDF: {TEST_PDF_PATH}")
    try:
        # Instantiate GeminiClient (required by FinancialDocParser)
        gemini_client = GeminiClient()

        # Instantiate FinancialDocParser
        doc_parser = FinancialDocParser(gemini_client=gemini_client)

        # Open the PDF file in binary read mode and read into a buffer
        with open(TEST_PDF_PATH, 'rb') as f:
            pdf_buffer = io.BytesIO(f.read()) # Create a buffer from the file content

        print(f"\nProcessing PDF: {TEST_PDF_PATH}")

        # Run the parser on the PDF buffer
        parsing_result = doc_parser.parse_pdf_to_markdown(pdf_buffer)

        # Print the results
        print("\n--- Parsing Result ---")
        if parsing_result.get("error"):
            print(f"Parsing failed: {parsing_result['error']}")
        else:
            print(f"Parsing successful. Pages processed: {parsing_result.get('page_count')}\n\n")
            # Print only the first 1000 characters of the markdown to avoid flooding output
            markdown_content = parsing_result.get('markdown_content', '')
            print(markdown_content[:1000] + ('...' if len(markdown_content) > 1000 else ''))
            if len(markdown_content) > 1000:
                 print(f"(Full markdown content is {len(markdown_content)} characters long)")


    except ValueError as e:
        print(f"Configuration Error: {e}") # e.g., API key missing
    except Exception as e:
        print(f"An unexpected error occurred during PDF parsing test: {e}")

Found test PDF: ../data/source_pdfs/10k_tesla_3_pages.pdf
Initializing Gemini client with API key: AIz...yQ

Processing PDF: ../data/source_pdfs/10k_tesla_3_pages.pdf
PDF has 3 pages
Rendering page 1/3
Rendering page 2/3
Rendering page 3/3
Starting page annotation with max 3 concurrent workers...
Page 1: Annotating (Attempt 1/6)
Page 2: Annotating (Attempt 1/6)
Page 3: Annotating (Attempt 1/6)
Page 2: Annotation successful.
Page 1: Annotation successful.
Page 3: Annotation successful.

--- Parsing Result ---
Parsing successful. Pages processed: 3


--- Page 1 Start ---

# UNITED STATES
# SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K

(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2021
OR
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _________ to _________

Commission File Number: 001-34756

## Tesla, In

In [4]:
from IPython.display import display, Markdown, Latex
print(f"\n(Full markdown content is {len(markdown_content)} characters long)\n\n")
display(Markdown(markdown_content))



(Full markdown content is 11494 characters long)




--- Page 1 Start ---

# UNITED STATES
# SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K

(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2021
OR
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _________ to _________

Commission File Number: 001-34756

## Tesla, Inc.

(Exact name of registrant as specified in its charter)
Delaware
(State or other jurisdiction of incorporation or organization)
91-2197729
(I.R.S. Employer Identification No.)
13101 Tesla Road
Austin, Texas
(Address of principal executive offices)
78725
(Zip Code)
(512) 516-8177
(Registrant's telephone number, including area code)

## Securities registered pursuant to Section 12(b) of the Act:

| Title of each class | Trading Symbol(s) | Name of each exchange on which registered |
| --- | --- | --- |
| Common stock | TSLA | The Nasdaq Global Select Market |

## Securities registered pursuant to Section 12(g) of the Act:

None

Indicate by check mark whether the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act. Yes ☐ No ☑

Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or 15(d) of the Act. Yes ☐ No ☑

Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 (“Exchange Act”) during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days. Yes ☑ No ☐

Indicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (§232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files). Yes ☑ No ☐

Indicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company. See the definitions of “large accelerated filer,” “accelerated filer,” “smaller reporting company” and “emerging growth company” in Rule 12b-2 of the Exchange Act:

Large accelerated filer ☑
Accelerated filer ☐
Non-accelerated filer ☐
Smaller reporting company ☐
Emerging growth company ☐

If an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. ☐

--- Page 1 End ---



--- Page 2 Start ---

# TESLA, INC.
## ANNUAL REPORT ON FORM 10-K FOR THE YEAR ENDED DECEMBER 31, 2021

### INDEX

|                                                            | Page |
| :--------------------------------------------------------- | :--- |
| **PART I.**                                                |      |
| Item 1.       Business                                       | 4    |
| Item 1A.      Risk Factors                                   | 14   |
| Item 1B.      Unresolved Staff Comments                      | 28   |
| Item 2.       Properties                                     | 28   |
| Item 3.       Legal Proceedings                              | 28   |
| Item 4.       Mine Safety Disclosures                        | 28   |
| **PART II.**                                               |      |
| Item 5.       Market for Registrant's Common Equity, Related Stockholder Matters and Issuer Purchases of Equity     Securities  | 29   |
| Item 6.       [Reserved]                                    | 30   |
| Item 7.       Management's Discussion and Analysis of Financial Condition and Results of Operations | 31   |
| Item 7A.      Quantitative and Qualitative Disclosures about Market Risk | 43   |
| Item 8.       Financial Statements and Supplementary Data   | 45   |
| Item 9.       Changes in and Disagreements with Accountants on Accounting and Financial Disclosure | 96   |
| Item 9A.      Controls and Procedures                        | 96   |
| Item 9B.      Other Information                              | 96   |
| Item 9C.      Disclosure Regarding Foreign Jurisdictions that Prevent Inspections | 96   |
| **PART III.**                                              |      |
| Item 10.      Directors, Executive Officers and Corporate Governance | 97   |
| Item 11.      Executive Compensation                         | 97   |
| Item 12.      Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters | 97   |
| Item 13.      Certain Relationships and Related Transactions, and Director Independence | 97   |
| Item 14.      Principal Accountant Fees and Services         | 97   |
| **PART IV.**                                               |      |
| Item 15.      Exhibits and Financial Statement Schedules    | 98   |
| Item 16.      Summary                                        | 115  |
| Signatures                                                 | 116  |

--- Page 2 End ---



--- Page 3 Start ---

# Note 4 – Goodwill and Intangible Assets

Goodwill decreased $7 million within the automotive segment from $207 million as of December 31, 2020 to $200 million as of December 31, 2021. There were no accumulated impairment losses as of December 31, 2021 and 2020.

Information regarding our intangible assets including assets recognized from our acquisitions was as follows (in millions):

|                                    | Gross Carrying Amount | Accumulated Amortization | Other | Net Carrying Amount | Gross Carrying Amount | Accumulated Amortization | Other | Net Carrying Amount |
| :--------------------------------- | :-------------------- | :------------------------- | :---- | :------------------ | :-------------------- | :------------------------- | :---- | :------------------ |
| **Finite-lived intangible assets:** |                       |                            |       |                     |                       |                            |       |                     |
| Developed technology              | $ 299                 | $ (150 )                   | $ 3   | $ 152               | $ 302                 | $ (111 )                   | $ 3   | $ 194               |
| Trade names                        | 2                     | (1 )                       | -     | 1                   | 3                     | (1 )                       | -     | 2                   |
| Favorable contracts and leases, net | 113                   | (40 )                       | -     | 73                  | 113                   | (32 )                       | -     | 81                  |
| Other                              | 36                    | (21 )                       | 1     | 16                  | 38                    | (18 )                       | 1     | 21                  |
| Total finite-lived intangible assets | 450                   | (212 )                      | 4     | 242                 | 456                   | (162 )                      | 4     | 298                 |
| **Indefinite-lived intangible assets:** |                       |                            |       |                     |                       |                            |       |                     |
| Gigafactory Nevada water rights    | 15                    | -                          | -     | 15                  | 15                    | -                          | -     | 15                  |
| Total intangible assets             | $ 465                 | $ (212 )                   | $ 4   | $ 257               | $ 471                 | $ (162 )                   | $ 4   | $ 313               |

Amortization expense during the years ended December 31, 2021, 2020 and 2019 was $51 million, $51 million and $44 million, respectively.

Total future amortization expense for finite-lived intangible assets was estimated as follows (in millions):

|       |       |
| :---- | :---- |
| 2022  | $ 49  |
| 2023  | 43    |
| 2024  | 28    |
| 2025  | 28    |
| 2026  | 28    |
| Thereafter | 66    |
| Total | $ 242 |

# Note 5 – Fair Value of Financial Instruments

ASC 820, *Fair Value Measurements*, states that fair value is an exit price, representing the amount that would be received to sell an asset or paid to transfer a liability in an orderly transaction between market participants. As such, fair value is a market-based measurement that should be determined based on assumptions that market participants would use in pricing an asset or a liability. The three-tiered fair value hierarchy, which prioritizes which inputs should be used in measuring fair value, is comprised of: (Level I) observable inputs such as quoted prices in active markets; (Level II) inputs other than quoted prices in active markets that are observable either directly or indirectly and (Level III) unobservable inputs for which there is little or no market data. The fair value hierarchy requires the use of observable market data when available in determining fair value. Our assets and liabilities that were measured at fair value on a recurring basis were as follows (in millions):

|                                                         | Fair Value | December 31, 2021                                     |
| :------------------------------------------------------ | :--------- | :---------------------------------------------------- |
|                                                         |            | Level I      | Level II | Level III |
| Money market funds (cash and cash equivalents)          | $ 9,548    | $ 9,548      | $ -      | $ -       |
| Corporate debt securities (short-term marketable securities) | 131        | -            | 131      | -         |
| Interest rate swap liabilities                          | 31         | -            | 31       | -         |
| Total                                                   | $ 9,710    | $ 9,548      | $ 162    | $ -       |
|                                                         | Fair Value | December 31, 2020                                     |
|                                                         |            | Level I      | Level II | Level III |
| Money market funds (cash and cash equivalents)          | $ 13,847   | $ 13,847     | $ -      | $ -       |
|                                                         | -          | -            | -        | -         |
|                                                         | 58         | -            | 58       | -         |
| Total                                                   | $ 13,905   | $ 13,847     | $ 58     | $ -       |

All of our money market funds were classified within Level I of the fair value hierarchy because they were valued using quoted prices in active markets. Our marketable securities are classified within Level II of the fair value hierarchy and the market approach was used to determine fair value of these investments. Our interest rate swaps were classified within Level II of the fair value hierarchy because they were valued using alternative pricing sources or models that utilized market observable inputs, including current and forward interest rates.

70

--- Page 3 End ---